**Exemplo de Streaming Para Leitura de Arquivos de Um Diretório**

In [1]:
#cria a seção a ser utiliza para estabelecer a conexão 
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode
from pyspark.sql.functions import split

spark = SparkSession \
    .builder \
    .appName("StructuredNetworkWordCount") \
    .getOrCreate()

23/01/26 00:11:04 WARN Utils: Your hostname, Deboras-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.68.104 instead (on interface en0)
23/01/26 00:11:04 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/01/26 00:11:04 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/01/26 00:11:05 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/01/26 00:11:05 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
23/01/26 00:11:05 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
23/01/26 00:11:05 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.


In [2]:
#cria o dataframe que será responsável por ler cada uma das linhas recebidas dos arquivos adicionados no diretório
files_dir = spark.readStream\
    .format("text")\
    .option("inferSchema", "true")\
    .option("maxFilesPerTrigger", 1)\
    .load("./data/arquivos/*.txt")

In [3]:
#verifica se criou o streaming
files_dir.isStreaming

True

In [4]:
# Divide as linhas recebidas em cada palavra
words = files_dir.select(
   explode(
       split(files_dir.value, " ")
   ).alias("word")
)

# cria o novo dataframe a ser responsável por contar a quantidade de palavras digitadas
wordCounts = words.groupBy("word").count()

In [5]:
#ordena as palavras que mais aparecem
from pyspark.sql.functions import desc
wordCounts = wordCounts.sort(desc("count"))

In [6]:
# Define a consulta (query) e como deve ser realizada a saída (sink) para o stream criado 
query = wordCounts \
    .writeStream \
    .outputMode("complete") \
    .format("console") \
    .start() #inicia a "query"


query.awaitTermination() #aguarda até que a "streaming query" termine 

23/01/26 00:13:12 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /private/var/folders/hn/zqmlf1cd4s3_llkx270x0kw80000gn/T/temporary-e5d19c22-b685-469e-917c-aa457cf35a2e. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
23/01/26 00:13:12 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 0
-------------------------------------------
+----------+-----+
|      word|count|
+----------+-----+
|     tinha|    7|
|       uma|    7|
|   caminho|    6|
|      meio|    6|
|        do|    6|
|     pedra|    5|
|        no|    5|
|esquecerei|    2|
|    pedra.|    2|
|        me|    2|
|     Nunca|    2|
|    minhas|    1|
|      vida|    1|
|       tão|    1|
|        de|    1|
|       que|    1|
|        No|    1|
|   retinas|    1|
|fatigadas.|    1|
|          |    1|
+----------+-----+
only showing top 20 rows



-------------------------------------------
Batch: 1
-------------------------------------------
+-------+-----+
|   word|count|
+-------+-----+
|    uma|    9|
|     do|    8|
|  tinha|    7|
|    que|    7|
|       |    7|
|caminho|    6|
|   meio|    6|
|    não|    6|
|  pedra|    5|
|     no|    5|
|     me|    4|
|  vasto|    3|
|    meu|    3|
|     de|    3|
|     eu|    3|
|  atrás|    3|
|     se|    3|
|     na|    3|
|      é|    2|
|  Mundo|    2|
+-------+-----+
only showing top 20 rows



-------------------------------------------
Batch: 2
-------------------------------------------
+-------+-----+
|   word|count|
+-------+-----+
|    não|   16|
|      e|   12|
|    que|   12|
|       |   12|
|   você|   11|
|    uma|    9|
|      a|    9|
|     se|    9|
|     do|    8|
|      o|    7|
|  tinha|    7|
|    sem|    7|
|     no|    7|
|caminho|    6|
|   meio|    6|
|     de|    6|
| agora,|    6|
|  José?|    5|
|  pedra|    5|
|      é|    4|
+-------+-----+
only showing top 20 rows



-------------------------------------------
Batch: 3
-------------------------------------------
+-------+-----+
|   word|count|
+-------+-----+
|    que|   18|
|    não|   18|
|      e|   13|
|       |   12|
|   você|   11|
|    uma|    9|
|      a|    9|
|     se|    9|
|      o|    8|
|  tinha|    8|
|     do|    8|
|     de|    7|
|    sem|    7|
|     no|    7|
|   para|    6|
|caminho|    6|
|   meio|    6|
|  amava|    6|
| agora,|    6|
|  José?|    5|
+-------+-----+
only showing top 20 rows



ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 